


# www.goodreads.com web scraper

The initial set up (placeholder) of this repository was forked from CS410Assignments/CourseProject, holder for UIUC CS 410 - Text Transformation Systems, Fall 2021, Final Project.

I will write a web scaper with Python to scrape Goodreads website, using Selenium and BeautifulSoup

**Overview** 
<br />
<br />This Project is a GoodReads website scraper that take all the newreleases by month and genre in last 2 years from the website and gather the information about them and display the 5 highest rating ones.

The content can be used for book recommandations and annual blog posts.

**Methods**
<br />
<br />The Web scaraper was written with:
- Python
- on JupiterNotbook
- Using Selenium and Beautifulsoup

Presentation is done with:

- HTML
- D3.js 

### The website for the Visualization
https://yasiss.github.io/CourseProject_GoodreadsScraper/


In [1]:
#Package everything together
#1
from selenium.webdriver.common.by import By
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup

import pandas as pd

import requests
from urllib.parse import urljoin


#Ctrl+] to indent
def login():

    url      = 'https://www.goodreads.com/'
    username = 'youremail@hotmail.com'
    password = '*********'
    driver.get(url)
    time.sleep(5)

    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.ESCAPE)
    time.sleep(1)

    #driver.find_element(By.CLASS_NAME,'gr-hyperlink').click()
    driver.find_element(By.LINK_TEXT, 'Sign In').click()
    time.sleep(1)

    driver.find_element(By.ID,'user_email').send_keys(username)
    time.sleep(1)
    driver.find_element(By.ID,'user_password').send_keys(password)
    time.sleep(1)

    driver.find_element(By.NAME,'next').click()

In [2]:
#2
serv = Service("C:/Users/yasi_/Downloads/chromedriver_win32/chromedriver.exe")

driver = webdriver.Chrome(service = serv)

login()

In [3]:
#3
import numpy as np

def process_books(html,y):
    #books = soup.find_all('div',{'class':'cover'}) #find the whole page info
    genres = soup.find_all('tr',{'class':'genre'}) #find the whole page info
    #genres = soup.find_all('div',{'class':'genre_releases'})
    clean_book_list = []
    
    
    for genre in genres:        

        books = genre.find_all('div',{'class':'book'})
        for book in books:
            data_dict = {}
            names_genres = genre.find('div',{'class':'name'})
            data_dict['book_url'] = book.a['href'] #we get the url and break it down and put in the dictionary
            data_dict['title'] = book.a['title'].strip()
            data_dict['genre'] = names_genres.find('h3').getText()
            data_dict['author'] = ''
            data_dict['rating'] = ''
            data_dict['pages'] = ''
            data_dict['description'] = ''
            data_dict['year'] = y
            data_dict['published'] = ''
   
          #  data_dict['img_src'] = book.img['src']

            clean_book_list.append(data_dict)
        #break  #just firsr record
    return clean_book_list

In [4]:
master_list_books = []

for j in range(2020,2022):
    print('Year: ', j)
    for i in range(1, 13):
        print('Month: ', i) #it starts from 0
        driver.get(f'https://www.goodreads.com/new_releases/{j}/{i}?tab=by_genre')
    
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        clean_book_list = process_books(html,j)

        master_list_books.extend(clean_book_list)

Year:  2020
Month:  1
Month:  2
Month:  3
Month:  4
Month:  5
Month:  6
Month:  7
Month:  8
Month:  9
Month:  10
Month:  11
Month:  12
Year:  2021
Month:  1
Month:  2
Month:  3
Month:  4
Month:  5
Month:  6
Month:  7
Month:  8
Month:  9
Month:  10
Month:  11
Month:  12


In [5]:
#5 Create a dataframe for our data
df_books = pd.DataFrame(master_list_books)

In [6]:
len(master_list_books)

1220

In [7]:
#### Scraping information of each book using for loop ####

#creating a loop counter
i = 0

#Loop through all 50 books
for url in df_books.iloc[:, 0]:
    
    #connect to url page
    note_resp = requests.get(url)
    
    #checking if the request is successful
    if note_resp.status_code == 200:
        #print("URL{}: {}".format(i+1, url))
        pass
    else:
        #print('Status code{}: Skipping URL #{}: {}'.format(note_resp.status_code, i+1, url))
        i = i+1
        continue
    
    #get HTML from url page
    note_html = note_resp.content
    
    #create beautifulsoup object for url page
    note_soup = BeautifulSoup(note_html, 'html.parser')
    
    #Extract Author particulars
    author_divs = note_soup.find_all('div', {'class':'authorName__container'})
    try:
        author_text = author_divs[0].find_all('a')[0].text
    except IndexError:
        author_text = 'Nil'
    
    #Extract rating particulars
    rating_divs = note_soup.find_all("div", {"class": "uitext stacked", "id": "bookMeta"})
    try:
        rating_text = rating_divs[0].find_all("span", {"itemprop": "ratingValue"})[0].text.strip()
    except IndexError:
        rating_text = 0 
    
    #Extracting page particulars
    page_divs = note_soup.find_all("div", {"class": "row"})
    try:
        page_text = page_divs[0].find_all("span", {"itemprop": "numberOfPages"})[0].text.strip(' pages')
    except IndexError:
        page_text = 0
    
    # When the book was published
    publish_divs = note_soup.find_all("div", {"class": "uitext darkGreyText", "id": "details"})
    try:
        publish_text = publish_divs[0].find_all("div", {"class": "row"})[1].text.strip()
        publish_text = publish_text.replace('\n',' ')
    except IndexError:
        publish_text = 'Nil'

    
    #Extracting description particulars
    description_divs = note_soup.find_all("div", {"class": "readable stacked", "id": "description"})
    try:
        description_text = description_divs[0].find_all("span")[1].text
    except IndexError:
        try:
            description_text = description_divs[0].find_all("span")[0].text
        except IndexError:
            description_text = "Nil"
    #book_description.append(description_text)
    
    #adding to our dataframe
    df_books.at[i,'author'] = author_text    
    df_books.at[i,'rating'] = float(rating_text)   
    df_books.at[i,'pages'] = page_text
    df_books.at[i,'description'] = description_text.strip()
    df_books.at[i,'published'] = publish_text.strip()
    
    #Incremeting the loop counter
    i = i+1
    #if i == 3 : break
    if i in (100,200,300,400,500,600, 700, 800, 900, 1000, 1100, 1200):
        print('No of books scraped so far: ', i)

No of books scraped so far:  100
No of books scraped so far:  200
No of books scraped so far:  300
No of books scraped so far:  400
No of books scraped so far:  500
No of books scraped so far:  600
No of books scraped so far:  700
No of books scraped so far:  800
No of books scraped so far:  1000
No of books scraped so far:  1100
No of books scraped so far:  1200


In [ ]:
df_books.iloc[:10]

In [10]:
#there were some blanks in data that prevented the sort and groupby to work
df_books['rating'].replace('', 0.0, inplace=True)
df_books['pages'].replace('', 0.0, inplace=True)

sorted_book_df = df_books.sort_values(['year','genre','rating'],ascending=(False, True, False))
sorted_book_df.reset_index(drop = True, inplace = True)

In [44]:
#top_book_df = sorted_book_df.groupby('year','genre','rating').head(5)
tmp_book_df = sorted_book_df.groupby(['year','genre']).head(5) #.nlargest(5)
tmp_book_df.reset_index(drop = True, inplace = True)

In [45]:
top_book_df = tmp_book_df.drop(columns='description')

In [50]:
top_book_df.head(5)

,book_url,title,genre,author,rating,pages,year,published
0,https://www.goodreads.com/book/show/57178191-5...,5 Worlds Book 5: The Emerald Gate,children's,Mark Siegel,4.67,272,2021,Published November 16th 2021 ...
1,https://www.goodreads.com/book/show/53420150-c...,Change Sings: a Children's Anthem,children's,Amanda Gorman,4.60,32,2021,Published September 21st 2021 ...
2,https://www.goodreads.com/book/show/55977822-w...,We Shall Overcome,children's,Bryan Collier,4.59,40,2021,Expected publication: December 28th 20...
3,https://www.goodreads.com/book/show/57375666-d...,Dancing with Daddy,children's,Anitra Rowe Schulte,4.51,40,2021,Published December 1st 2021 b...
4,https://www.goodreads.com/book/show/53410874-b...,Barefoot Dreams of Petra Luna,children's,Alda P. Dobbs,4.49,288,2021,Published September 14th 2021 ...


In [48]:
top_book_df.to_csv(".../TopFivePerGenre.csv", index = False)

In [11]:
sorted_book_df.to_csv(".../NewReleases.csv")